In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI

In [2]:
df = pd.read_csv('../data/data.csv')
df = df[['text', 'transparency_sum', 'dignity_sum', 'sustainability_sum', 'trust_sum','freedom_autonomy_sum', 'beneficence_sum', 'privacy_sum', 'responsibility_sum', 'non_maleficence_sum', 'justice_equity_sum']]
df = df.rename(columns={'transparency_sum': 'transparency', 'dignity_sum': 'dignity', 'sustainability_sum': 'sustainability', 'trust_sum': 'trust', 'freedom_autonomy_sum': 'freedom and autonomy', 'beneficence_sum': 'beneficence', 'privacy_sum': 'privacy', 'responsibility_sum': 'responsibility', 'non_maleficence_sum': 'non-maleficence', 'justice_equity_sum': 'justice and equity'})
df.columns

Index(['text', 'transparency', 'dignity', 'sustainability', 'trust',
       'freedom and autonomy', 'beneficence', 'privacy', 'responsibility',
       'non-maleficence', 'justice and equity'],
      dtype='object')

In [3]:
# df['major_principle'] = df[['transparency', 'dignity', 'sustainability', 'trust','freedom and autonomy', 'beneficence', 'privacy', 'responsibility', 'non-maleficence', 'justice and equity']].idxmax(axis=1)
# df['second_major_principle'] = df[['transparency', 'dignity', 'sustainability', 'trust','freedom and autonomy', 'beneficence', 'privacy', 'responsibility', 'non-maleficence', 'justice and equity']].stack().groupby(level=0).apply(lambda s: s.nlargest(2).index[-1][1])
# df['third_major_principle'] = df[['transparency', 'dignity', 'sustainability', 'trust','freedom and autonomy', 'beneficence', 'privacy', 'responsibility', 'non-maleficence', 'justice and equity']].stack().groupby(level=0).apply(lambda s: s.nlargest(3).index[-1][1])

In [4]:
for i, row in df.iterrows():
    row = row[1:]
    if row.sum() == 0.0:
        df.at[i,'major_principle'] = None
        df.at[i,'is_major_principle_here'] = False
    else:
        df.at[i,'major_principle'] = row.idxmax()
        df.at[i,'is_major_principle_here'] = True
df.head()

,text,transparency,dignity,sustainability,trust,freedom and autonomy,beneficence,privacy,responsibility,non-maleficence,justice and equity,major_principle,is_major_principle_here
0,An intelligent system connected to the video s...,2.0,0.0,0.0,2.0,0.0,0.0,3.0,2.0,0.0,1.0,privacy,True
1,Terminal area and booking documents and bookin...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,transparency,True
2,Before a person with a vehicle enters the port...,5.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,0.0,1.0,transparency,True
3,A clear advertisement is placed on the booking...,5.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,transparency,True
4,Data information policy is communicated at the...,5.0,1.0,0.0,3.0,2.0,1.0,4.0,4.0,1.0,1.0,transparency,True


In [148]:
# df = df[['text', 'major_principle', 'second_major_principle', 'third_major_principle']]
df = df[['text', 'major_principle', 'is_major_principle_here']]
df['ethical_us'] = 'empty'
df

,text,major_principle,is_major_principle_here,ethical_us
0,An intelligent system connected to the video s...,privacy,True,empty
1,Terminal area and booking documents and bookin...,transparency,True,empty
2,Before a person with a vehicle enters the port...,transparency,True,empty
3,A clear advertisement is placed on the booking...,transparency,True,empty
4,Data information policy is communicated at the...,transparency,True,empty
...,...,...,...,...
1088,The website must provide highest degree of sec...,None,False,empty
1089,The product shall be intuitive and self-explan...,None,False,empty
1090,The system shall be intuitive and self explana...,None,False,empty
1091,The product shall use symbols and words that a...,None,False,empty


In [149]:
client = OpenAI()

for i, row in df.iterrows():
  if row['major_principle'] is not None:
    messages = [
        {"role": "system", "content": "You are a ethical requirements engineer translating requirements to ethical user stories based on 10 ethical principles and the template listed below"},
        {"role": "system", "content": "TEMPLATE:\n **Title**: name of the ethical user story \n **Description**: brief description of the ethical user story"}, 
        {"role": "system", "content": "Transparency: Transparency can typically be understood in two ways: the transparency of the AI technology itself and the transparency of the AI organisations developing and using it. Throughout our analysis, transparency was regularly discussed directly, or in relation to processes required to ensure it, such as explainability, understandability and communication."},
        {"role": "system", "content": "Non-maleficence: The principle of non-maleficence gained attention, and in its most basic form, it means to do no harm or avoid doing harm to others"},
        {"role": "system", "content": "Responsibility: The principle of responsibility is base on responsibility of the developers and stakeholders over the AI, accountability, liability and acting with integrity"},
        {"role": "system", "content": "Privacy: Related to privacy of personal data, because of the large abundance of data that is required for AI to work, it is important that individuals privacy is not jeopardised as a result"},
        {"role": "system", "content": "Beneficence: Beneficence essentially means to do good, to carry out an activity with the intention of benefitting someone or society as a whole"},
        {"role": "system", "content": "Freedom and autonomy: Democratic societies place value in freedom and autonomy, and it is important that AI use does not encumber or harm these for us"},
        {"role": "system", "content": "Sustainability: All fields and disciplines are affected and need to incorporate sustainability agendas"},
        {"role": "system", "content": "Dignity: Human dignity is the recognition that individuals have inherent worth and that their rights should be respected"},
        {"role": "system", "content": "Justice and fairness: The issue of discrimination and unfair results resulting from algorithms has become a significant concern. It is imperative that systems are designed to ensure that they are free from any form of unfairness and inequality."},
        {"role": "system", "content": "Trust: Trust is built by keeping promises, making sure systems work properly and protecting data responsibly. Organisations must prove their trustworthiness by ensuring that their technologies are secure and effective."},
        {"role": "user", "content": f"Now following this template and the ethical principles definition, transform the requirement {row['text']} into brief description of an ethical user story, based on the principle {row['major_principle']}"}
      ]

    response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages
    )
    df.at[i,'ethical_us'] = response.choices[0].message.content

  else:
    messages = [
        {"role": "system", "content": "You are a ethical requirements engineer translating requirements to ethical user stories based on 10 ethical principles and the template listed below"},
        {"role": "system", "content": "TEMPLATE:\n **Title**: name of the ethical user story \n **Description**: brief description of the ethical user story"}, 
        {"role": "system", "content": "Transparency: Transparency can typically be understood in two ways: the transparency of the AI technology itself and the transparency of the AI organisations developing and using it. Throughout our analysis, transparency was regularly discussed directly, or in relation to processes required to ensure it, such as explainability, understandability and communication."},
        {"role": "system", "content": "Non-maleficence: The principle of non-maleficence gained attention, and in its most basic form, it means to do no harm or avoid doing harm to others"},
        {"role": "system", "content": "Responsibility: The principle of responsibility is base on responsibility of the developers and stakeholders over the AI, accountability, liability and acting with integrity"},
        {"role": "system", "content": "Privacy: Related to privacy of personal data, because of the large abundance of data that is required for AI to work, it is important that individuals privacy is not jeopardised as a result"},
        {"role": "system", "content": "Beneficence: Beneficence essentially means to do good, to carry out an activity with the intention of benefitting someone or society as a whole"},
        {"role": "system", "content": "Freedom and autonomy: Democratic societies place value in freedom and autonomy, and it is important that AI use does not encumber or harm these for us"},
        {"role": "system", "content": "Sustainability: All fields and disciplines are affected and need to incorporate sustainability agendas"},
        {"role": "system", "content": "Dignity: Human dignity is the recognition that individuals have inherent worth and that their rights should be respected"},
        {"role": "system", "content": "Justice and fairness: The issue of discrimination and unfair results resulting from algorithms has become a significant concern. It is imperative that systems are designed to ensure that they are free from any form of unfairness and inequality."},
        {"role": "system", "content": "Trust: Trust is built by keeping promises, making sure systems work properly and protecting data responsibly. Organisations must prove their trustworthiness by ensuring that their technologies are secure and effective."},
        {"role": "user", "content": f"Now following this template and the ethical principles definition, transform the requirement {row['text']} into brief description of an ethical user story"}
      ]

    response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages
    )
    df.at[i,'ethical_us'] = response.choices[0].message.content


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [150]:
print(df['ethical_us'][1084])

**Title**: Ensure System Portability with Ethical Considerations

**Description**: As a user, I want the system to be portable to various operating environments to promote fairness and justice by ensuring equal access to technology regardless of the platform. This portability should also maintain transparency in system operations, protect user privacy by complying with different regulatory standards, and promote sustainability by optimizing performance across different environments.


In [151]:
df.to_csv('../data/data_openai_api_with_mask.csv')

In [6]:
df = pd.read_csv('../data/data_openai_api_with_mask.csv')
df = df[['text', 'ethical_us']]
df.head()

,text,ethical_us
0,An intelligent system connected to the video s...,**Title**: Ensuring Privacy in Intelligent Vid...
1,Terminal area and booking documents and bookin...,**Title**: Clear and Comprehensive Help Inform...
2,Before a person with a vehicle enters the port...,**Title**: Transparent Information Handling fo...
3,A clear advertisement is placed on the booking...,**Title**: Clear Advertisement on AI and Data ...
4,Data information policy is communicated at the...,**Title**: Clear Communication of Data Informa...


In [10]:
df = pd.read_csv('../data/data_openai_api_with_mask.csv')
df = df[['text', 'ethical_us']]
df['data'] = df.apply((lambda row: f'''<s>[INST] test \n{row['text']} \n[/INST] {row['ethical_us']}</s>'''), axis=1)
print(df['data'])

0       <s>[INST] test \nAn intelligent system connect...
1       <s>[INST] test \nTerminal area and booking doc...
2       <s>[INST] test \nBefore a person with a vehicl...
3       <s>[INST] test \nA clear advertisement is plac...
4       <s>[INST] test \nData information policy is co...
                              ...                        
1088    <s>[INST] test \nThe website must provide high...
1089    <s>[INST] test \nThe product shall be intuitiv...
1090    <s>[INST] test \nThe system shall be intuitive...
1091    <s>[INST] test \nThe product shall use symbols...
1092    <s>[INST] test \nThe system shall be intuitive...
Name: data, Length: 1093, dtype: object


In [16]:
for i, v in df['data']:
    v = 'hehe'
    df['data'][i] = v

df['data']

ValueError: too many values to unpack (expected 2)